In [1]:
import torch, os
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from utils import depth_to_rgb
from loss import MultiViewPhotometricLoss
from dataset import ImageDataset
from depth_resnet import DepthResNet

In [2]:
torch.cuda.empty_cache()

In [4]:
train_dir = 'datasets/3rdfloor'
train_dataset = ImageDataset(train_dir, 'images', 'poses',
                             'intrinsics.txt', context=1, stride=10, step=1)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [5]:
step = 0
device = torch.device('cuda')
model = DepthResNet(version='50pt').to(device)

for name, param in model.named_parameters():
    param.requires_grad = True

In [6]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                             lr=0.0001)

photometric_loss = MultiViewPhotometricLoss(num_scales=4, ssim_loss_weight=0.85,
                                            smooth_loss_weight=0.1)

In [7]:
run_name = '3rdfloor'
exp_dir = os.path.join('runs', run_name)
writer = SummaryWriter(exp_dir)

In [8]:
print_step = 10

for epoch in range(10000):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        original_image, context_images, trans_matrix, intrinsics = data
        original_image = original_image.to(device)
        context_images = [ci.to(device) for ci in context_images]
        trans_matrix = [mat.to(device) for mat in trans_matrix]
        
        optimizer.zero_grad()

        preds = model(original_image)
        output = photometric_loss(original_image, context_images, preds,
                                  intrinsics, trans_matrix)
        loss = output['loss']
        warped_context = output['warped_context']

        loss.backward()
        optimizer.step()
        
        step += 1
        running_loss += loss.item()
        if step % print_step == 0:
            print('Epoch/step: {}/{}'.format(epoch, step))
            writer.add_scalar('loss',
                            running_loss / print_step,
                            step)
            vis_image = torch.stack([original_image[0],
                                    depth_to_rgb(preds[0][0]),
                                    context_images[0][0],
                                    warped_context[0][0][0]], 0)
            vis_image = F.interpolate(vis_image, scale_factor=0.5)
            writer.add_images('original_invdepth_context_warped',
                              vis_image,
                              global_step=step)
            running_loss = 0.0

Epoch/step: 0/10


/home/arvrlab/.local/lib/python3.6/site-packages/torch/nn/functional.py:3500: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "


Epoch/step: 0/20
Epoch/step: 0/30
Epoch/step: 0/40
Epoch/step: 0/50
Epoch/step: 0/60
Epoch/step: 0/70
Epoch/step: 0/80
Epoch/step: 0/90
Epoch/step: 0/100
Epoch/step: 1/110
Epoch/step: 1/120
Epoch/step: 1/130
Epoch/step: 1/140
Epoch/step: 1/150
Epoch/step: 1/160
Epoch/step: 1/170
Epoch/step: 1/180
Epoch/step: 1/190
Epoch/step: 1/200
Epoch/step: 2/210
Epoch/step: 2/220
Epoch/step: 2/230
Epoch/step: 2/240
Epoch/step: 2/250
Epoch/step: 2/260
Epoch/step: 2/270
Epoch/step: 2/280
Epoch/step: 2/290
Epoch/step: 2/300
Epoch/step: 2/310
Epoch/step: 3/320
Epoch/step: 3/330
Epoch/step: 3/340
Epoch/step: 3/350
Epoch/step: 3/360
Epoch/step: 3/370
Epoch/step: 3/380
Epoch/step: 3/390
Epoch/step: 3/400
Epoch/step: 3/410
Epoch/step: 4/420
Epoch/step: 4/430
Epoch/step: 4/440
Epoch/step: 4/450
Epoch/step: 4/460
Epoch/step: 4/470
Epoch/step: 4/480
Epoch/step: 4/490
Epoch/step: 4/500
Epoch/step: 4/510
Epoch/step: 4/520
Epoch/step: 5/530
Epoch/step: 5/540
Epoch/step: 5/550
Epoch/step: 5/560
Epoch/step: 5/570


Epoch/step: 40/4230
Epoch/step: 40/4240
Epoch/step: 40/4250
Epoch/step: 40/4260
Epoch/step: 41/4270
Epoch/step: 41/4280
Epoch/step: 41/4290
Epoch/step: 41/4300
Epoch/step: 41/4310
Epoch/step: 41/4320
Epoch/step: 41/4330
Epoch/step: 41/4340
Epoch/step: 41/4350
Epoch/step: 41/4360
Epoch/step: 42/4370
Epoch/step: 42/4380
Epoch/step: 42/4390
Epoch/step: 42/4400
Epoch/step: 42/4410
Epoch/step: 42/4420
Epoch/step: 42/4430
Epoch/step: 42/4440
Epoch/step: 42/4450
Epoch/step: 42/4460
Epoch/step: 42/4470
Epoch/step: 43/4480
Epoch/step: 43/4490
Epoch/step: 43/4500
Epoch/step: 43/4510
Epoch/step: 43/4520
Epoch/step: 43/4530
Epoch/step: 43/4540
Epoch/step: 43/4550
Epoch/step: 43/4560
Epoch/step: 43/4570
Epoch/step: 44/4580
Epoch/step: 44/4590
Epoch/step: 44/4600
Epoch/step: 44/4610
Epoch/step: 44/4620
Epoch/step: 44/4630
Epoch/step: 44/4640
Epoch/step: 44/4650
Epoch/step: 44/4660
Epoch/step: 44/4670
Epoch/step: 44/4680
Epoch/step: 45/4690
Epoch/step: 45/4700
Epoch/step: 45/4710
Epoch/step: 45/4720


Epoch/step: 80/8330
Epoch/step: 80/8340
Epoch/step: 80/8350
Epoch/step: 80/8360
Epoch/step: 80/8370
Epoch/step: 80/8380
Epoch/step: 80/8390
Epoch/step: 80/8400
Epoch/step: 80/8410
Epoch/step: 80/8420
Epoch/step: 81/8430
Epoch/step: 81/8440
Epoch/step: 81/8450
Epoch/step: 81/8460
Epoch/step: 81/8470
Epoch/step: 81/8480
Epoch/step: 81/8490
Epoch/step: 81/8500
Epoch/step: 81/8510
Epoch/step: 81/8520
Epoch/step: 82/8530
Epoch/step: 82/8540
Epoch/step: 82/8550
Epoch/step: 82/8560
Epoch/step: 82/8570
Epoch/step: 82/8580
Epoch/step: 82/8590
Epoch/step: 82/8600
Epoch/step: 82/8610
Epoch/step: 82/8620
Epoch/step: 82/8630
Epoch/step: 83/8640
Epoch/step: 83/8650
Epoch/step: 83/8660
Epoch/step: 83/8670
Epoch/step: 83/8680
Epoch/step: 83/8690
Epoch/step: 83/8700
Epoch/step: 83/8710
Epoch/step: 83/8720
Epoch/step: 83/8730
Epoch/step: 84/8740
Epoch/step: 84/8750
Epoch/step: 84/8760
Epoch/step: 84/8770
Epoch/step: 84/8780
Epoch/step: 84/8790
Epoch/step: 84/8800
Epoch/step: 84/8810
Epoch/step: 84/8820


...

Epoch/step: 619/64450
Epoch/step: 619/64460
Epoch/step: 619/64470
Epoch/step: 619/64480
Epoch/step: 620/64490
Epoch/step: 620/64500
Epoch/step: 620/64510
Epoch/step: 620/64520
Epoch/step: 620/64530
Epoch/step: 620/64540
Epoch/step: 620/64550
Epoch/step: 620/64560
Epoch/step: 620/64570
Epoch/step: 620/64580
Epoch/step: 621/64590
Epoch/step: 621/64600
Epoch/step: 621/64610
Epoch/step: 621/64620
Epoch/step: 621/64630
Epoch/step: 621/64640
Epoch/step: 621/64650
Epoch/step: 621/64660
Epoch/step: 621/64670
Epoch/step: 621/64680
Epoch/step: 622/64690
Epoch/step: 622/64700
Epoch/step: 622/64710
Epoch/step: 622/64720
Epoch/step: 622/64730
Epoch/step: 622/64740
Epoch/step: 622/64750
Epoch/step: 622/64760
Epoch/step: 622/64770
Epoch/step: 622/64780
Epoch/step: 622/64790
Epoch/step: 623/64800
Epoch/step: 623/64810
Epoch/step: 623/64820
Epoch/step: 623/64830
Epoch/step: 623/64840
Epoch/step: 623/64850
Epoch/step: 623/64860
Epoch/step: 623/64870
Epoch/step: 623/64880
Epoch/step: 623/64890
Epoch/step

Epoch/step: 655/68180
Epoch/step: 655/68190
Epoch/step: 655/68200
Epoch/step: 655/68210
Epoch/step: 655/68220
Epoch/step: 656/68230
Epoch/step: 656/68240
Epoch/step: 656/68250
Epoch/step: 656/68260
Epoch/step: 656/68270
Epoch/step: 656/68280
Epoch/step: 656/68290
Epoch/step: 656/68300
Epoch/step: 656/68310
Epoch/step: 656/68320
Epoch/step: 657/68330
Epoch/step: 657/68340
Epoch/step: 657/68350
Epoch/step: 657/68360
Epoch/step: 657/68370
Epoch/step: 657/68380
Epoch/step: 657/68390
Epoch/step: 657/68400
Epoch/step: 657/68410
Epoch/step: 657/68420
Epoch/step: 657/68430
Epoch/step: 658/68440
Epoch/step: 658/68450
Epoch/step: 658/68460
Epoch/step: 658/68470
Epoch/step: 658/68480
Epoch/step: 658/68490
Epoch/step: 658/68500
Epoch/step: 658/68510
Epoch/step: 658/68520
Epoch/step: 658/68530
Epoch/step: 659/68540
Epoch/step: 659/68550
Epoch/step: 659/68560
Epoch/step: 659/68570
Epoch/step: 659/68580
Epoch/step: 659/68590
Epoch/step: 659/68600
Epoch/step: 659/68610
Epoch/step: 659/68620
Epoch/step

Epoch/step: 691/71910
Epoch/step: 691/71920
Epoch/step: 691/71930
Epoch/step: 691/71940
Epoch/step: 691/71950
Epoch/step: 691/71960
Epoch/step: 692/71970
Epoch/step: 692/71980
Epoch/step: 692/71990
Epoch/step: 692/72000
Epoch/step: 692/72010
Epoch/step: 692/72020
Epoch/step: 692/72030
Epoch/step: 692/72040
Epoch/step: 692/72050
Epoch/step: 692/72060
Epoch/step: 692/72070
Epoch/step: 693/72080
Epoch/step: 693/72090
Epoch/step: 693/72100
Epoch/step: 693/72110
Epoch/step: 693/72120
Epoch/step: 693/72130
Epoch/step: 693/72140
Epoch/step: 693/72150
Epoch/step: 693/72160
Epoch/step: 693/72170
Epoch/step: 694/72180
Epoch/step: 694/72190
Epoch/step: 694/72200
Epoch/step: 694/72210
Epoch/step: 694/72220
Epoch/step: 694/72230
Epoch/step: 694/72240
Epoch/step: 694/72250
Epoch/step: 694/72260
Epoch/step: 694/72270
Epoch/step: 694/72280
Epoch/step: 695/72290
Epoch/step: 695/72300
Epoch/step: 695/72310
Epoch/step: 695/72320
Epoch/step: 695/72330
Epoch/step: 695/72340
Epoch/step: 695/72350
Epoch/step

Epoch/step: 727/75640
Epoch/step: 727/75650
Epoch/step: 727/75660
Epoch/step: 727/75670
Epoch/step: 727/75680
Epoch/step: 727/75690
Epoch/step: 727/75700
Epoch/step: 727/75710
Epoch/step: 728/75720
Epoch/step: 728/75730
Epoch/step: 728/75740
Epoch/step: 728/75750
Epoch/step: 728/75760
Epoch/step: 728/75770
Epoch/step: 728/75780
Epoch/step: 728/75790


KeyboardInterrupt: 

In [10]:
checkpoint = { 
        'model': model.state_dict(),
        'step': step}

torch.save(checkpoint, 'checkpoint_'+run_name+'.pth')